### Model Training¶

In [1]:
#!pip install sklearn

In [2]:
#!pip install -U scikit-learn

In [3]:
## Import Required Packages:

## Basic Imports
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Modelling

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings


In [4]:
## Import csv data as a pandas DataFrame
os.getcwd()
os.listdir()

['.ipynb_checkpoints',
 'catboost_info',
 'data',
 'EDA_Student_Performance.ipynb',
 'Model_Training.ipynb']

In [5]:
df = pd.read_csv('data/stud.csv')

In [6]:
## Show top 5 records
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [7]:
## preparing X and Y variables:

X=df.drop(columns=['math_score'],axis=1)
X.shape

(1000, 7)

In [8]:
print("Categories in 'gender' variable:     ",end=" " )
print(df['gender'].unique())

print("Categories in 'race_ethnicity' variable:  ",end=" ")
print(df['race_ethnicity'].unique())

print("Categories in'parental level of education' variable:",end=" " )
print(df['parental_level_of_education'].unique())

print("Categories in 'lunch' variable:     ",end=" " )
print(df['lunch'].unique())

print("Categories in 'test preparation course' variable:     ",end=" " )
print(df['test_preparation_course'].unique())

Categories in 'gender' variable:      ['female' 'male']
Categories in 'race_ethnicity' variable:   ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in'parental level of education' variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' variable:      ['standard' 'free/reduced']
Categories in 'test preparation course' variable:      ['none' 'completed']


In [9]:
y = df['math_score']

In [10]:
from sklearn.compose import ColumnTransformer

In [11]:
## Create ColumnTransformer

num_features=X.select_dtypes(exclude='object').columns
cat_features=X.select_dtypes(include='object').columns

from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder()

preprocessor=ColumnTransformer(
    [
        ('OneHotEncoder',oh_transformer,cat_features),
        ('StandardScaler',numeric_transformer,num_features),
    ]



)

In [12]:
X.head(2)

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88


In [13]:
X=preprocessor.fit_transform(X)


In [14]:
X.shape

(1000, 19)

In [15]:
## Separate Dataset into train and test

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,y_train.shape

((800, 19), (800,))

In [22]:
### Create an Evaluate Function to give all metrics after model Training

def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2=r2_score(true,predicted)
    return mae,mse,rmse,r2
    

In [32]:
models={
    'Linear Regression':LinearRegression(),
    'Lasso Regressor':Lasso(),
    'Ridge Regressor':Ridge(),
    'DecisionTree Regressor':DecisionTreeRegressor(),
    'XGB Regressor':XGBRegressor(),
    'AdaBoost Regressor':AdaBoostRegressor(),
    'CatBoosting Regressor':CatBoostRegressor(verbose=False),
    'RandomForest Regressor':RandomForestRegressor(),
    "K-Neighbors Regressor": KNeighborsRegressor()

    
}

model_list=[]
r2_list=[]


for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train) ## Train model
    ## Make Predictions
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae ,model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae ,model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Squared Error: {:.4f}".format(model_train_mse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.1}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Squared Error: {:.4f}".format(model_test_mse))
    print("- Mean Absolute Error: {:.4f}",(model_test_mae))
    print("- R2 Score: {:}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Model performance for Training set
- Root Mean Squared Error: 5.3402
- Mean Squared Error: 28.5176
- Mean Absolute Error: 4.2723
- R2 Score: 0.9
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.4214
- Mean Squared Error: 29.3913
- Mean Absolute Error: {:.4f} 4.2253125
- R2 Score: 0.8792164757750934


Model performance for Training set
- Root Mean Squared Error: 6.5938
- Mean Squared Error: 43.4784
- Mean Absolute Error: 5.2063
- R2 Score: 0.8
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.5197
- Mean Squared Error: 42.5064
- Mean Absolute Error: {:.4f} 5.157881810347763
- R2 Score: 0.8253197323627853


Model performance for Training set
- Root Mean Squared Error: 5.3233
- Mean Squared Error: 28.3378
- Mean Absolute Error: 4.2650
- R2 Score: 0.9
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3904
- Mean Squared Error: 29.0563
- Mean Absolute Error

In [33]:
print(r2_list)

[0.8792164757750934, 0.8253197323627853, 0.8805931485028738, 0.743505340710714, 0.8215894515233413, 0.8500313103434279, 0.8516318920747058, 0.8502033289997477, 0.7838129945787431]


In [34]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Models','R2_Score']).sort_values(by='R2_Score',ascending=False)

,Models,R2_Score
2,Ridge Regressor,0.880593
0,Linear Regression,0.879216
6,CatBoosting Regressor,0.851632
7,RandomForest Regressor,0.850203
5,AdaBoost Regressor,0.850031
1,Lasso Regressor,0.825320
4,XGB Regressor,0.821589
8,K-Neighbors Regressor,0.783813
3,DecisionTree Regressor,0.743505
